In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using SiLQR

In [ ]:
"""
Solver Setup
"""

# Define bouncing quadrotor model
# Crazyflie 2.1 parameters: https://arxiv.org/pdf/1608.05786
e = 1.0
g = 9.81
m = 0.027
j = [1.436e-5, 1.395e-5,  2.173e-5]
b = 7.9379e-12 / 3.1582e-10 * ones(4)
c = 0.0283 * ones(4)
d = 0.0283 * ones(4)
system = bouncing_quadrotor(e, g, m, j, b, c, d)

# Stage and terminal costs
Q = 1e-4 * diagm([2e2*ones(3); 1e2*ones(4); ones(6)])
R = 1e-4 * I(system.nu)
Qf = 1e2 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 100
Δt = 0.01
params = SiLQR.Parameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory
pref = [2.0, 1.0, 0.2]
qref = [1.0; zeros(3)]
xref = [pref; qref; zeros(6)]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]

# Initial conditions
p0 = [0.0, 0.0, 1.0]
q0 = qref #random_unit_quat()
v0 = [0.0, 0.0, 0.0]
ω0 = zeros(3)
params.x0 = [p0; q0; v0; ω0]
params.mI = :down

nothing

In [ ]:
"""
Solve using SiLQR
"""

# Solve
sol = SiLQR.Solution(params)
cache = SiLQR.Cache(params)
@time SiLQR.solve!(sol, cache, params; max_iter=20)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1,3), xs; xlim=(0,3), ylim=(0,3))

nothing